In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='4,5,6,7'

In [ ]:
%run -p ../train_cnf_disentangle_rl_semisup.py --data cifar10 --dims 64,64,64 --strides 1,1,1,1 --num_blocks 2 --layer_type concat --multiscale True --rademacher True --batch_size 900 --test_batch_size 500 --save ../experiments_published/infocnf_cond_cifar10_bs900_sratio_0_5_drop_0_5_rl_stdscale_6_nsup4k_beta_1_run1 --seed 1 --lr 0.001 --conditional True --controlled_tol False --train_mode semisup --log_freq 10 --weight_y 0.5 --condition_ratio 0.5 --dropout_rate 0.5 --scale_fac 1.0 --scale_std 6.0 --labeled_batch_size 450 --num_train_sup 4000 --beta 1.0
#

/tancode/repos/tan-ffjord/train_cnf_disentangle_rl_semisup.py
import argparse
import os
import time
import numpy as np

import torch
import torch.optim as optim
import torchvision.datasets as dset
import torchvision.transforms as tforms
from torchvision.utils import save_image

import torch.utils.data as data
from torch.utils.data import Dataset

import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 300

from PIL import Image
import os.path
import errno
import codecs

import lib.layers as layers
import lib.utils as utils
import lib.multiscale_parallel as multiscale_parallel
import lib.modules as modules
import lib.thops as thops

from train_misc import standard_normal_logprob
from train_misc import set_cnf_options, count_nfe, count_parameters, count_total_time, count_nfe_gate
from train_misc import add_spectral_norm, spectral_norm_power_iteration
from train_misc import create_regularization_fns, get_regularization, append_regularization_to_log

from tensorboardX import SummaryWr

Load semi-sup datasets for CIFAR10


ODENVP(
  (transforms): ModuleList(
    (0): StackedCNFLayers(
      (chain): ModuleList(
        (0): LogitTransform()
        (1): CNF_Gate(
          (gate_net): FeedforwardGateI(
            (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
            (conv1): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu1): ReLU(inplace)
            (conv2): Conv2d(10, 10, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
            (bn2): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu2): ReLU(inplace)
            (avg_layer): AdaptiveAvgPool2d(output_size=(1, 1))
            (linear_layer): Conv2d(10, 1, kernel_size=(1, 1), stride=(1, 1))
            (sigmoid): Sigmoid()
          )
          (odefunc): RegularizedODEfunc(
            (odefunc): ODEfu

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Epoch 0001 | Time 245.4578, Epoch Time 1602.0221(1602.0221), Bit/dim 7.0173(best: inf), Bit/dim Marginal 7.0170(best: inf), Xent 2.0832, Loss 8.0589, Error 0.7076(best: inf)
===> Using batch size 900. Total 102 iterations/epoch.
Iter 0110 | Time 14.1054(14.1421) | Bit/dim 6.9789(7.3713) | Xent 2.0808(2.0931) | Loss 18.1506(19.3598) | Error 0.6956(0.7078) Steps 580(522.03) | Grad Norm 8.7332(6.0048) | Total Time 0.00(0.00)
Iter 0120 | Time 14.1051(14.1004) | Bit/dim 6.9533(7.2658) | Xent 2.0297(2.0841) | Loss 17.8148(18.9692) | Error 0.6867(0.7050) Steps 532(526.34) | Grad Norm 3.5184(6.0687) | Total Time 0.00(0.00)
Iter 0130 | Time 14.2262(14.1030) | Bit/dim 6.9220(7.1794) | Xent 2.0018(2.0728) | Loss 17.7786(18.6684) | Error 0.6711(0.7005) Steps 550(531.60) | Grad Norm 4.4391(6.6068) | Total Time 0.00(0.00)
Iter 0140 | Time 14.6627(14.1497) | Bit/dim 6.8954(7.1065) | Xent 1.9783(2.0604) | Loss 17.7073(18.4328) | Error 0.6378(0.6932) Steps 538(536.10) | Grad Norm 4.1642(6.7688) | Total

...

Epoch 0009 | Time 98.7162, Epoch Time 1784.6145(1616.4068), Bit/dim 4.4050(best: 4.5358), Bit/dim Marginal 4.3730(best: 4.4963), Xent 1.7452, Loss 5.2776, Error 0.6166(best: 0.6418)
===> Using batch size 900. Total 102 iterations/epoch.
Iter 0920 | Time 15.9254(16.0065) | Bit/dim 4.3766(4.4273) | Xent 1.2991(1.3331) | Loss 11.7343(12.6567) | Error 0.4778(0.4716) Steps 586(628.12) | Grad Norm 38.8532(21.5168) | Total Time 0.00(0.00)
Iter 0930 | Time 16.2319(16.0282) | Bit/dim 4.4242(4.4156) | Xent 1.2864(1.3258) | Loss 11.8562(12.4189) | Error 0.4511(0.4679) Steps 628(628.71) | Grad Norm 26.1960(20.9651) | Total Time 0.00(0.00)
Iter 0940 | Time 16.5154(16.0674) | Bit/dim 4.3793(4.4157) | Xent 1.2863(1.3212) | Loss 11.8966(12.2768) | Error 0.4489(0.4663) Steps 622(630.34) | Grad Norm 25.4156(22.1420) | Total Time 0.00(0.00)
Iter 0950 | Time 14.9045(15.9794) | Bit/dim 4.3663(4.4043) | Xent 1.1622(1.3196) | Loss 11.6039(12.1320) | Error 0.4356(0.4672) Steps 598(628.63) | Grad Norm 14.0440(

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Epoch 0010 | Time 88.0825, Epoch Time 1806.8662(1622.1206), Bit/dim 4.2981(best: 4.4050), Bit/dim Marginal 4.2724(best: 4.3730), Xent 1.7261, Loss 5.1611, Error 0.5934(best: 0.6166)
===> Using batch size 900. Total 102 iterations/epoch.
Iter 1030 | Time 15.9009(16.0119) | Bit/dim 4.3100(4.3204) | Xent 1.1633(1.2472) | Loss 11.3895(12.1710) | Error 0.4222(0.4451) Steps 628(632.61) | Grad Norm 12.6601(17.5085) | Total Time 0.00(0.00)
Iter 1040 | Time 16.3854(16.0333) | Bit/dim 4.3164(4.3108) | Xent 1.3173(1.2485) | Loss 11.5361(11.9950) | Error 0.4467(0.4451) Steps 592(629.72) | Grad Norm 28.0277(18.7422) | Total Time 0.00(0.00)
Iter 1050 | Time 15.7213(16.0398) | Bit/dim 4.2923(4.3036) | Xent 1.3699(1.2511) | Loss 11.6477(11.8863) | Error 0.4667(0.4431) Steps 604(628.98) | Grad Norm 25.2426(19.7142) | Total Time 0.00(0.00)
Iter 1060 | Time 15.9821(16.0667) | Bit/dim 4.2777(4.2962) | Xent 1.1806(1.2507) | Loss 11.5313(11.7892) | Error 0.4333(0.4428) Steps 640(631.57) | Grad Norm 11.5891(

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Epoch 0011 | Time 89.4173, Epoch Time 1872.8471(1629.6424), Bit/dim 4.2035(best: 4.2981), Bit/dim Marginal 4.1838(best: 4.2724), Xent 1.6498, Loss 5.0284, Error 0.5775(best: 0.5934)
===> Using batch size 900. Total 102 iterations/epoch.
Iter 1130 | Time 15.6760(16.2040) | Bit/dim 4.1719(4.2205) | Xent 1.1804(1.1746) | Loss 11.0512(11.9221) | Error 0.3756(0.4159) Steps 598(630.45) | Grad Norm 21.3853(15.7718) | Total Time 0.00(0.00)
Iter 1140 | Time 17.3350(16.2714) | Bit/dim 4.1940(4.2096) | Xent 1.0813(1.1640) | Loss 11.2135(11.7323) | Error 0.3578(0.4139) Steps 658(630.99) | Grad Norm 11.8528(14.7893) | Total Time 0.00(0.00)
Iter 1150 | Time 16.2126(16.2598) | Bit/dim 4.1643(4.2001) | Xent 1.0834(1.1451) | Loss 11.0789(11.5601) | Error 0.3867(0.4061) Steps 652(629.96) | Grad Norm 10.4499(14.1400) | Total Time 0.00(0.00)
Iter 1160 | Time 15.6558(16.3555) | Bit/dim 4.1678(4.1928) | Xent 1.0565(1.1385) | Loss 11.1465(11.4648) | Error 0.3844(0.4049) Steps 604(632.51) | Grad Norm 24.6771(